### Notas de Análise da Educação Brasileira

> *Como se divide a cobertura da educação básica entre os tipos de escola Federal, Estadual, Municipal, e Privada?*

*Trazendo as ferramentas necessárias*

In [45]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

*Importando a base de dados*

In [40]:
dados2022 = pd.read_csv('/content/drive/MyDrive/dados/microdados_ed_basica_2022.csv', sep=';', encoding='latin-1')
dados2022.head(5)

<ipython-input-40-4029b3fafa47>:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  dados2022 = pd.read_csv('/content/drive/MyDrive/dados/microdados_ed_basica_2022.csv', sep=';', encoding='latin-1')


,NU_ANO_CENSO,NO_REGIAO,CO_REGIAO,NO_UF,SG_UF,CO_UF,NO_MUNICIPIO,CO_MUNICIPIO,NO_MESORREGIAO,CO_MESORREGIAO,...,QT_TUR_FUND_AF,QT_TUR_MED,QT_TUR_PROF,QT_TUR_PROF_TEC,QT_TUR_EJA,QT_TUR_EJA_FUND,QT_TUR_EJA_MED,QT_TUR_ESP,QT_TUR_ESP_CC,QT_TUR_ESP_CE
0,2022,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,2,...,0.0,0.0,0.0,0.0,10.0,6.0,4.0,3.0,3.0,0.0
2,2022,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,2,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
4,2022,Norte,1,Rondônia,RO,11,Alta Floresta D'Oeste,1100015,Leste Rondoniense,2,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0


De acordo ao Dicionário de Dados disponível no conjunto de dados do Censo Escolar 2022 do INEP, as colunas que vamos precisar olhar para responder essa pergunta é:

- TP_DEPENDENCIA: Que descreve a Dependência Administrativa e tem como referência os valores (1 - Federal, 2 - Estadual, 3 - Municipal, 4 - Privada);

- QT_MAT_BAS: Que descreve o Número de Matrículas na Educação Básica;

Dessa forma, para respondermos a nossa pergunta, fazemos:

In [41]:
dados2022[['QT_MAT_BAS', 'TP_DEPENDENCIA']].groupby(['TP_DEPENDENCIA']).sum().reset_index()

,TP_DEPENDENCIA,QT_MAT_BAS
0,1,380605.0
1,2,14761395.0
2,3,23240028.0
3,4,9000046.0


Agora vamos substituir os valores de referência das dependências pelo seus significados:

In [44]:
'O total de matrículas em 2022 foi de: ' + str(int(dados2022['QT_MAT_BAS'].sum()))

'O total de matrículas em 2022 foi de: 47382074'

Para calcularmos o percetual de matrículas em cada dependência administrativa, fazemos:

In [13]:
(dados2022[['QT_MAT_BAS', 'TP_DEPENDENCIA']].groupby('TP_DEPENDENCIA').sum().reset_index() / dados2022['QT_MAT_BAS'].sum()) * 100.

,TP_DEPENDENCIA,QT_MAT_BAS
0,0.000002,0.803268
1,0.000004,31.153966
2,0.000006,49.048144
3,0.000008,18.994622
